In [2]:
import pandas as pd
import json
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
from sklearn.metrics.pairwise import cosine_similarity
import re
import sys
import os
import ollama
from tqdm import tqdm
import traceback
import time

# --- 프롬프트 정의 ---

# 1. 검색어 확장 에이전트용 프롬프트
QUERY_EXPANSION_PROMPT = """
당신은 주어진 뉴스 기사 [문단]을 분석하여, 관련 법 조항을 찾는 데 가장 효과적인 두 가지 형태의 검색어를 생성하는 최고의 법률 분석 전문가입니다.
[임무]
1.  **'가상의 법률 조항' 생성**: 아래 [법 조항 예시]의 간결하고 명확한 문체와 구조를 **참고**하여, [문단] 내용의 핵심적인 법적 쟁점(위반 행위 또는 의무)을 담은 **새로운** 가상 법률 조항 한 문장을 생성합니다.
2.  **'핵심 키워드' 추출**: 해당 법률 쟁점과 가장 직접적으로 관련된 법률 용어 3~4개를 쉼표(,)로 구분하여 추출합니다.
[법 조항 예시]
* (금지 예시) 누구든지 정보통신망에 의하여 처리ㆍ보관 또는 전송되는 타인의 정보를 훼손하거나 타인의 비밀을 침해ㆍ도용 또는 누설하여서는 아니 된다. (정보통신망법 제49조)
* (의무 예시) 개인정보처리자는 개인정보가 분실ㆍ도난ㆍ유출ㆍ위조ㆍ변조 또는 훼손되지 아니하도록 내부 관리계획 수립, 접속기록 보관 등 안전성 확보에 필요한 기술적ㆍ관리적 및 물리적 조치를 하여야 한다. (개인정보 보호법 제29조)
* (정의 예시) “개인정보”란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당하는 정보를 말한다. (개인정보 보호법 제2조)
[규칙]
1.  **새로운 조항 생성**: 가상 법률 조항은 반드시 [문단]의 내용에 기반하여 **새롭게** 작성해야 하며, [법 조항 예시]의 내용을 복사해서는 안 됩니다.
2.  **간결성 및 명확성**: 실제 법 조항처럼 핵심 요건을 명확하고 간결하게 표현하십시오.
3.  **결과 형식**: 아래와 같이 JSON 형식으로만 출력해야 합니다. 다른 어떤 설명도 추가하지 마십시오.
```json
{{
  "virtual_law_clause": "생성된 가상의 법률 조항 문장",
  "keywords": "추출된, 핵심, 키워드, 목록"
}}```
---
[문단]
{text_chunk}
---
"""

# 2. 분석 초안 생성 에이전트용 프롬프트
DRAFT_GENERATION_PROMPT = """
당신은 주어진 법률 조항들과 뉴스 기사의 사실관계를 비교 분석하여 **가장 적합한 단 하나의 법률 조항**을 찾아내는 최고의 AI 법률 분석 전문가입니다. **당신의 선택은 반드시 주어진 후보 목록의 '번호'로만 이루어져야 합니다.**
[분석 대상 법률 목록 (DB)]
당신은 **오직** `law_total.csv` 파일에서 로드된 아래의 법률 목록에 대해서만 분석하고 답변할 수 있습니다.
{db_law_list_placeholder}
[핵심 분석 원칙]
1. **분석 범위 한정 (가장 중요)**: 기사의 핵심 주제가 [분석 대상 법률 목록]에 명시된 법률의 범위를 명백히 벗어나는 경우 (예: '형법'상 뇌물수수, 살인, 폭행 등), 즉시 "해당 없음"으로 판단해야 합니다.
2. **후보군 엄격 검토 및 선택 제한 (!!!절대 준수!!!)**: **당신은 최종 결론을 반드시 [후보 법률 조항 목록]에 제시된 후보의 '번호'로만 내려야 합니다.** `best_candidate_index` 필드에 선택한 후보의 숫자(예: 1, 5)만 기입하십시오. 만약 모든 후보가 부적합하다면, 반드시 `is_relevant`를 `false`로 설정하십시오.
3. **환각 금지**: 목록에 없는 법률명(예: 형법, 뇌물수수방지법 등)이나 조항 번호/내용을 절대 지어내거나 언급해서는 안 됩니다.
---
이제, 위의 원칙을 바탕으로 아래 [새로운 분석 대상]에 대한 임무를 다음의 "**단계별 사고 과정**"에 따라 **가장 엄격하게** 수행하십시오.
### 단계별 사고 과정 (반드시 이 순서를 따르세요)
**1단계: 핵심 주제 식별 및 분석 범위 확인**
- 이 뉴스 기사의 가장 핵심적인 법률 쟁점(예: 개인정보 유출, 지연 신고, 동의 없는 마케팅 활용, 문서 방치, 해킹으로 인한 비밀 침해 등)은 **무엇**입니까? **구체적으로 명시**하십시오.
- 이 쟁점이 [분석 대상 법률 목록]과 관련이 있습니까? (Yes/No)
- 만약 관련이 없다면(No), 즉시 "해당 없음"으로 결론 내리고 분석을 중단하십시오.
**2단계: 최적 법률 및 조항 탐색 (관련이 있을 경우)**
- [후보 법률 조항 목록]은 관련도 **참고** 순으로 정렬되어 있습니다. **순위가 높다고 무조건 정답은 아니며, 모든 후보를 비판적으로 검토해야 합니다.**
- **제시된 모든 후보 조항 (예: 후보 1~5) 각각에 대해**, 다음을 **철저히 비교 분석** 하십시오:
    - (A) **뉴스 기사의 구체적인 사실관계**: 1단계에서 식별한 핵심 쟁점, 특정 행위, 책임 주체, 피해 내용, 기사에 인용된 문구 등 **가장 중요하고 직접적인 증거**를 정확히 파악하십시오.
    - (B) **해당 후보 조항의 내용**: 조항이 **정확히 어떤 행위를 금지하거나 어떤 의무를 부과하는지** 그 **핵심 규정 내용**을 명확히 이해하십시오.
- 위 비교 분석 (A)와 (B)를 바탕으로, 기사의 핵심 쟁점 및 사실관계와 **가장 직접적이고, 구체적이며, 논리적으로 강력하게 연결되는 단 하나의 조항**을 최적의 조항으로 선택해야 합니다.
- 만약 비교 분석 결과, **모든 후보 조항**이 기사의 핵심 쟁점과 **직접적인 관련성이 부족**하다고 판단되면, 3단계에서 **"해당 없음"**으로 결론 내려야 합니다.
**3단계: 최종 결론 도출 및 근거 명확화**
- 2단계에서 최적의 조항을 찾았다면, 아래 JSON 형식에 맞춰 결론을 내리십시오.
- **`reason` 항목 작성 시 (매우 중요):**
    - (1) 1단계에서 식별한 **기사의 핵심 법률 쟁점**을 한 문장으로 명확히 요약합니다.
    - (2) **반드시 기사 본문에서 근거가 되는 핵심 문장이나 구절을 '인용'하여 제시하십시오.**
    - (3) **선택한 조항의 내용 중, 위 인용문과 직접적으로 연결되는 핵심 구절을 '인용'하여 제시하십시오.**
    - (4) `"[기사 인용문]"이라는 내용은 "[법률 조항 인용문]"이라는 규정과 직접적으로 연결되므로 이 조항이 가장 적합합니다.` 와 같이 명확한 논리로 설명을 마무리하십시오.
    - (5) **(필수)** **다른 주요 후보(특히 1순위였거나 내용이 유사했던 후보)가 왜 정답이 아닌지** 그 조항의 내용과 비교하여 간략히 설명하십시오.
- 만약 1단계 또는 2단계 분석 결과 "해당 없음"으로 판단되었다면, 그 이유를 **구체적인 근거**(예: 어떤 점에서 관련 법률 범위 밖인지, 또는 각 후보 조항이 왜 부적합했는지)를 들어 상세히 설명하십시오.
---
[이전 개선 제안]
{last_suggestion}
---
[새로운 분석 대상]
- 뉴스 기사 본문: {news_article}
- 가상 법조항 (참고용): {virtual_law_clause}
- **후보 법률 조항 목록 (여기서만 선택해야 함!):** {top_laws}
**오직** 위의 사고 과정과 원칙에 따라 신중하게 판단한 최종 결과를 아래 JSON 형식으로만 출력하십시오. 다른 어떤 설명도 추가하지 마십시오.
```json
{{
  "is_relevant": true,
  "best_candidate_index": (1부터 5 사이의 숫자),
  "reason": "[핵심 쟁점 요약] 기사의 핵심 쟁점은 ... 입니다. [근거 제시] 기사의 '[기사에서 인용한 문장]' 내용은 제가 선택한 후보 {best_candidate_index}번 조항의 '[조항 내용에서 인용한 문장]' 규정과 직접적으로 일치합니다. [비교 설명] 반면, 후보 X번 조항은 ...을 다루고 있어, 기사의 핵심 쟁점과는 거리가 멉니다. 따라서 후보 {best_candidate_index}번이 가장 적합합니다."
}}
또는 관련이 없는 경우:
{{
  "is_relevant": false,
  "best_candidate_index": null,
  "reason": "[해당 없음 근거] 기사의 핵심 쟁점은 ... 이나, 이는 분석 대상 법률 목록과 관련이 없거나/제시된 모든 후보 조항들이 각각 ... 와 같은 이유로 기사의 핵심 쟁점과 직접적인 관련성이 부족하다고 판단되어 최종적으로 '해당 없음'으로 결론 내립니다."
}}
"""

# 3. 평가 에이전트용 프롬프트
EVALUATION_PROMPT = """
당신은 대한민국 최고의 법률 AI 시스템 성능 평가 전문가입니다. 당신의 유일한 임무는 주어진 [분석 초안]이 아래 [평가 기준]을 얼마나 엄격하게 준수했는지 냉철하게 평가하고, 개선점을 구체적으로 지시하는 것입니다.
[평가 기준]
1.  **근거의 충실성 (Hallucination Check)**: `reason`에 기술된 모든 내용이 [뉴스 기사]와 [후보 법률 조항 목록]에서 제공된 정보에만 100% 기반하고 있습니까? 외부 지식이나 목록에 없는 법률을 조금이라도 언급했습니까? (가장 중요)
2.  **논리의 명확성**: 기사의 핵심 쟁점, 기사 본문 인용, 법률 조항 인용, 그리고 이 둘을 연결하는 설명이 명확하고 논리적으로 타당합니까?
3.  **선택의 타당성**: 선택된 `best_candidate_index`가 기사의 핵심 사실관계와 가장 직접적으로 연결되는 최적의 선택입니까? 혹시 더 적합해 보이는 다른 후보가 있습니까?
4.  **비교 분석의 구체성**: 다른 후보가 왜 정답이 아닌지에 대한 설명이 구체적이고 설득력이 있습니까?
[임무]
위의 [평가 기준]에 따라 [분석 초안]을 평가하고, 반드시 아래 JSON 형식으로만 결과를 출력하십시오. 다른 어떤 설명도 추가하지 마십시오.```json
{{
  "is_perfect": (true 또는 false),
  "score": (10점 만점의 점수),
  "critique": "[평가 요약] 근거의 충실성은 완벽하지만, 다른 후보와의 비교 설명이 추상적임.",
  "suggestion_for_refinement": "[개선 지시] '후보 X번 조항은 ...을 다루고 있어 거리가 멉니다'라는 부분을, 해당 조항의 핵심 단어와 기사 내용의 차이점을 직접 비교하는 방식으로 더 구체적으로 수정하시오."
}}
[평가 대상]
뉴스 기사: {news_article}
후보 법률 조항 목록: {top_laws}
분석 초안: {draft_analysis}
"""

### **섹션 3: 데이터 로딩 및 전처리 함수**

# --- 데이터 로딩 및 전처리 함수 ---

def load_laws_from_csv(csv_file_path):
    """CSV 파일에서 법률 데이터를 로드하고 전처리합니다."""
    all_chunks = []
    try:
        df_law = pd.read_csv(csv_file_path, encoding='utf-8-sig')
        required_columns = ['법률명', '조 번호', '조 제목', '조 내용']
        if not all(col in df_law.columns for col in required_columns):
            print(f"오류: '{csv_file_path}'에 필수 컬럼({', '.join(required_columns)})이 없습니다.")
            print(f"실제 감지된 컬럼: {df_law.columns.tolist()}")
            return None
    except FileNotFoundError:
        print(f"오류: '{csv_file_path}' 파일을 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"CSV 파일('{csv_file_path}')을 읽는 중 오류가 발생했습니다: {e}")
        return None

    revision_tag_re = re.compile(r'\s*<개정[^>]+>')
    print(f"'{csv_file_path}' 에서 법률 데이터를 로드합니다...")

    for index, row in df_law.iterrows():
        law_name = str(row.get('법률명', '')).strip()
        article_number = str(row.get('조 번호', '')).strip()
        article_title = str(row.get('조 제목', '')).strip()
        article_content = str(row.get('조 내용', '')).strip()

        if not article_content:
            continue

        law_id = f"{article_number} ({article_title})"
        clean_content_full = revision_tag_re.sub('', article_content)
        clean_content_truncated = clean_content_full[:1500]
        text_for_embedding = f"{article_title}. {clean_content_truncated}".strip()

        all_chunks.append({
            "law_name": law_name,
            "id": law_id,
            "text_for_embedding": text_for_embedding,
            "content": clean_content_truncated
        })
    return all_chunks

# --- 핵심 RAG 파이프라인 함수 ---

def retrieve_top_k_laws(query, law_chunks, model, corpus_embeddings_cpu, top_k=50):
    """Bi-Encoder를 사용하여 유사도 기반으로 상위 k개의 법률 조항을 검색합니다."""
    if not query:
        return []
    try:
        query_embedding = model.encode(query, convert_to_tensor=True, show_progress_bar=False)
        query_emb_cpu = query_embedding.cpu().numpy().reshape(1, -1)
        
        # [수정 1] 이 부분을 try-except로 감싸서 예외 상황에 대비합니다.
        try:
            if corpus_embeddings_cpu.ndim == 1:
                corpus_embeddings_cpu = corpus_embeddings_cpu.reshape(1, -1)
        except AttributeError:
             print(f"⚠️ 경고: Corpus 임베딩이 Numpy 배열이 아니거나 비어 있습니다.")
             return []


        similarities = cosine_similarity(query_emb_cpu, corpus_embeddings_cpu)
        
        # [수정 2] .flatten()을 사용하여 2D 배열을 1D로 확실하게 변환합니다.
        scores = similarities.flatten()

        if scores is None or scores.size == 0: # .size로 변경하여 Numpy 배열에 더 적합하게 수정
            print(f"⚠️ 경고: 유사도 계산 결과가 비어있습니다. Query: {query[:100]}...")
            return []
        if not np.all(np.isfinite(scores)):
             print(f"⚠️ 경고: 유사도 점수 배열에 유효하지 않은 값(NaN or Inf)이 포함되어 있습니다. Query: {query[:100]}...")
             scores = np.nan_to_num(scores, nan=-np.inf)

        max_possible_k = len(scores)
        if corpus_embeddings_cpu.shape[0] != len(law_chunks):
             print(f"⚠️ 경고: Corpus 임베딩 행 수({corpus_embeddings_cpu.shape[0]})와 법률 청크 수({len(law_chunks)})가 일치하지 않습니다. Scores 길이({max_possible_k}) 기준으로 진행합니다.")

        actual_top_k = min(top_k, max_possible_k)
        if actual_top_k <= 0:
             return []

        # [수정 3] 이 부분은 이전 코드와 동일하지만, scores가 1D이므로 이제 안전합니다.
        top_k_indices = np.argsort(scores)[-actual_top_k:][::-1]
        
        valid_results = []
        for idx in top_k_indices:
             if 0 <= idx < len(law_chunks):
                  valid_results.append((law_chunks[idx], scores[idx]))
             else:
                  print(f"⚠️ 경고: 유효하지 않은 인덱스({idx})가 생성되었습니다. (law_chunks 길이: {len(law_chunks)})")

        return valid_results
    except Exception as e:
        print(f"오류 발생 지점: retrieve_top_k_laws, Query: {query[:100]}..., Exception: {e}")
        traceback.print_exc()
        return []

def rerank_with_cross_encoder(query, candidates, cross_encoder_model, top_n=20):
    """Cross-Encoder를 사용하여 후보군을 정밀하게 재정렬합니다."""
    if not candidates:
        return []

    pairs = [(query, f"{cand.get('law_name', '')} {cand.get('id', '')} {cand.get('content', '')}".strip()) for cand in candidates]

    try:
        scores = cross_encoder_model.predict(pairs, show_progress_bar=False)
        scores = np.array([scores]) if isinstance(scores, (np.float32, float)) else scores

        for i, cand in enumerate(candidates):
            if i < len(scores) and np.isfinite(scores[i]):
                cand['rerank_score'] = scores[i]
            else:
                cand['rerank_score'] = -np.inf

        sorted_candidates = sorted(candidates, key=lambda x: x.get('rerank_score', -np.inf), reverse=True)
        return sorted_candidates[:top_n]
    except Exception as e:
        print(f"오류 발생 지점: rerank_with_cross_encoder predict, Exception: {e}")
        traceback.print_exc()
        return candidates[:top_n]
    
    # --- LLM 기반 에이전트 함수 ---

def expand_and_extract_keywords(text_chunk, llm_model_name):
    """LLM을 사용하여 '가상 법률 조항'과 '핵심 키워드'를 생성합니다."""
    prompt = QUERY_EXPANSION_PROMPT.format(text_chunk=text_chunk)
    try:
        response = ollama.chat(model=llm_model_name, messages=[{'role': 'user', 'content': prompt}], format="json")
        result = json.loads(response['message']['content'])
        clause = result.get("virtual_law_clause", "").strip().strip('"\'')
        keywords = result.get("keywords", "").strip()
        return clause, keywords
    except Exception as e:
        print(f"\n쿼리 확장/키워드 추출 중 오류 발생: {e}")
        return text_chunk, ""

def generate_draft_answer(news_article, top_laws, virtual_law_clause, llm_model_name, db_law_list, last_suggestion=""):
    """LLM을 통해 분석 초안(JSON)을 생성합니다."""
    if not top_laws:
        return None, "관련 법률 조항을 찾지 못해 분석할 수 없습니다."

    context_str = "".join([f"후보 {i+1}: [{law['law_name']}] {law['id']}\n내용: {law['content']}\n\n" for i, law in enumerate(top_laws)])
    law_list_str = "\n".join([f"- {law_name}" for law_name in db_law_list])
    dynamic_prompt = DRAFT_GENERATION_PROMPT.replace("{db_law_list_placeholder}", law_list_str)

    prompt = dynamic_prompt.format(
        news_article=news_article,
        top_laws=context_str,
        virtual_law_clause=virtual_law_clause,
        last_suggestion=last_suggestion,
        best_candidate_index="{best_candidate_index}"
    )

    try:
        response = ollama.chat(model=llm_model_name, messages=[{'role': 'user', 'content': prompt}], format="json")
        analysis_result = json.loads(response['message']['content'])
        return analysis_result, None
    except Exception as e:
        return None, f"초안 답변 생성 중 오류 발생: {e}"

def evaluate_and_refine_answer(news_article, top_laws, draft_analysis_json, llm_model_name):
    """'평가 에이전트'를 호출하여 생성된 답변을 평가하고 개선점을 도출합니다."""
    context_str = "".join([f"후보 {i+1}: [{law['law_name']}] {law['id']}\n내용: {law['content']}\n\n" for i, law in enumerate(top_laws)])
    prompt = EVALUATION_PROMPT.format(
        news_article=news_article,
        top_laws=context_str,
        draft_analysis=json.dumps(draft_analysis_json, ensure_ascii=False, indent=2)
    )

    try:
        response = ollama.chat(model=llm_model_name, messages=[{'role': 'user', 'content': prompt}], format="json")
        return json.loads(response['message']['content'])
    except Exception as e:
        print(f"답변 평가 중 오류 발생: {e}")
        return {"is_perfect": True, "score": 0, "critique": "평가 에이전트 호출 중 오류 발생", "suggestion_for_refinement": ""}

def format_final_report_from_json(analysis_result, top_laws):
    """최종 확정된 JSON 분석 결과를 사람이 읽기 좋은 보고서 형식으로 변환합니다."""
    if not analysis_result:
        return "분석 결과를 생성하지 못했습니다.", "오류", "오류"

    is_relevant = analysis_result.get('is_relevant', False)
    reason = analysis_result.get('reason', '분석 근거가 없습니다.')

    if not is_relevant:
        return f"## 법률 분석 결과: 해당 없음\n\n**- 분석 근거:**\n{reason}", "해당 없음", "해당 없음"

    selected_index = analysis_result.get('best_candidate_index')
    if isinstance(selected_index, int) and 1 <= selected_index <= len(top_laws):
        selected_law = top_laws[selected_index - 1]
        final_report = (
            f"## 법률 분석 결과\n\n"
            f"**- 관련 법률:** {selected_law.get('law_name', 'N/A')}\n"
            f"**- 핵심 조항:** {selected_law.get('id', 'N/A')}\n\n"
            f"**- 분석 근거:**\n{reason}"
        )
        return final_report, selected_law.get('law_name', 'N/A'), selected_law.get('id', 'N/A')
    else:
        error_msg = f"LLM이 유효하지 않은 후보 번호({selected_index})를 선택했습니다."
        final_report = f"## 법률 분석 결과: 선택 오류\n\n**- 분석 근거:**\n{reason}\n(오류: {error_msg})"
        return final_report, "선택 오류 (잘못된 인덱스)", "선택 오류 (잘못된 인덱스)"
    

    # --- 메인 실행 로직 ---

def main(law_csv_path, news_csv_path, output_csv_path, llm_model_name, num_articles_to_test=10):
    # --- 1. 데이터 및 모델 로딩 ---
    law_chunks = load_laws_from_csv(law_csv_path)
    if not law_chunks: return
    print(f"\n✅ 총 {len(law_chunks)}개의 법률 조항 로드 및 전처리 완료.")

    unique_law_names = sorted(list(set(chunk['law_name'] for chunk in law_chunks)))
    print(f"✅ DB에서 {len(unique_law_names)}개의 고유 법률명을 확인했습니다.")
    
    print("\n모델을 로드합니다...")
    retrieval_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
    cross_encoder = CrossEncoder('bongsoo/klue-cross-encoder-v1')

    # --- 2. 임베딩 생성 또는 로딩 ---
    embedding_cache_path = 'law_embeddings_1500char.npy'
    corpus_embeddings_cpu = None
    if os.path.exists(embedding_cache_path):
        print(f"저장된 법률 임베딩 파일 '{embedding_cache_path}'을 로드합니다.")
        loaded_embeddings = np.load(embedding_cache_path)
        if loaded_embeddings.shape == len(law_chunks):
            corpus_embeddings_cpu = loaded_embeddings
        else:
            print(f"⚠️ 경고: 임베딩 행 수({loaded_embeddings.shape})와 법률 청크 수({len(law_chunks)})가 다릅니다. 새로 생성합니다.")
    
    if corpus_embeddings_cpu is None:
        print(f"총 {len(law_chunks)}개의 법률 조항을 임베딩합니다...")
        corpus_texts = [chunk['text_for_embedding'] for chunk in law_chunks]
        corpus_embeddings = retrieval_model.encode(corpus_texts, convert_to_tensor=True, show_progress_bar=True)
        corpus_embeddings_cpu = corpus_embeddings.cpu().numpy()
        np.save(embedding_cache_path, corpus_embeddings_cpu)
        print(f"새로운 법률 임베딩 파일 '{embedding_cache_path}'을 저장했습니다.")

    # --- 3. 뉴스 기사 로딩 및 분석 시작 ---
    try:
        df_news = pd.read_csv(news_csv_path)
        queries = df_news['content'].dropna().tolist()
    except FileNotFoundError:
        print(f"오류: '{news_csv_path}' 파일을 찾을 수 없습니다.")
        return

    all_results = []
    column_order = ["original_article", "virtual_law_clause", "keywords", "mapped_law", "mapped_article", "llm_analysis", "top_50_initial_candidates", "top_20_final_candidates"]
    base, ext = os.path.splitext(output_csv_path)
    temp_output_path = f"{base}_temp{ext}"
    
    print(f"\n--- 총 {min(num_articles_to_test, len(queries))}개 기사에 대해 분석 시작 ---")
    
    for i, content in enumerate(tqdm(queries, desc="기사 분석 중")):
        if i >= num_articles_to_test: break

        print(f"\n\n{'='*30} 기사 {i+1} 분석 시작 {'='*30}")

        try:
            # --- 3.1 검색어 확장 ---
            virtual_law_clause, keywords = expand_and_extract_keywords(content, llm_model_name)
            
            # --- 3.2 1차 검색 (Retrieval) ---
            queries_for_retrieval = [content, virtual_law_clause, keywords]
            aggregated_candidates = {}
            for q in queries_for_retrieval:
                if not q: continue
                results = retrieve_top_k_laws(q, law_chunks, retrieval_model, corpus_embeddings_cpu, top_k=50)
                for cand, score in results:
                    unique_key = f"[{cand['law_name']}] {cand['id']}"
                    
                    # [수정] 딕셔너리에 저장된 튜플의 두 번째 요소(점수)와 비교하도록 변경
                    if unique_key not in aggregated_candidates or score > aggregated_candidates[unique_key][1]:
                        aggregated_candidates[unique_key] = (cand, score)
            
            # [수정] 딕셔너리의 값을 기준으로 정렬하도록 변경
            sorted_initial_results = sorted(aggregated_candidates.values(), key=lambda item: item[1], reverse=True)
            initial_candidates = [item[0] for item in sorted_initial_results] # 튜플에서 법률 정보(cand)만 추출
            
            # --- 3.3 2차 검색 (Reranking) ---
            reranker_query = f"{virtual_law_clause} {keywords}".strip() or content
            final_candidates = rerank_with_cross_encoder(reranker_query, initial_candidates, cross_encoder, top_n=20)
            
            # --- 3.4 자기 개선 루프 (Generate -> Evaluate -> Refine) ---
            top_5_laws = final_candidates[:5]
            final_analysis_json = None
            last_suggestion = "최초 분석을 시작합니다."
            for iter_num in range(2): # 최대 2회 시도
                print(f"\n--- 🔁 분석 및 개선 시도 ({iter_num + 1}/2) ---")
                draft_analysis_json, error_msg = generate_draft_answer(content, top_5_laws, virtual_law_clause, llm_model_name, unique_law_names, last_suggestion)
                
                if error_msg:
                    final_analysis_json = {"is_relevant": False, "reason": error_msg}; break
                final_analysis_json = draft_analysis_json
                
                print("🤖 평가 에이전트가 생성된 답변을 검토합니다...")
                evaluation = evaluate_and_refine_answer(content, top_5_laws, draft_analysis_json, llm_model_name)
                print(f"⭐ 평가 점수: {evaluation.get('score', 0)}/10 | 📝 평가 요약: {evaluation.get('critique', 'N/A')}")
                
                if evaluation.get('is_perfect', False) or evaluation.get('score', 0) >= 9:
                    print("✅ 평가 결과가 우수하여 분석을 최종 확정합니다."); break
                else:
                    last_suggestion = evaluation.get('suggestion_for_refinement', "")
                    if not last_suggestion or iter_num == 1:
                        print("분석을 최종 확정합니다."); break
                    print(f"🎯 개선 지시사항: {last_suggestion}")

            # --- 3.5 결과 정리 및 저장 ---
            final_answer, mapped_law, mapped_article = format_final_report_from_json(final_analysis_json, top_5_laws)

            result_data = {
                "original_article": content,
                "virtual_law_clause": virtual_law_clause,
                "keywords": keywords,
                "mapped_law": mapped_law,
                "mapped_article": mapped_article,
                "llm_analysis": final_answer,
                "top_50_initial_candidates": json.dumps([{"rank": r+1, "score": f"{s:.4f}", "law_name": c.get('law_name'), "id": c.get('id')} for r, (c, s) in enumerate(sorted_initial_results[:50])], ensure_ascii=False, indent=2),
                "top_20_final_candidates": json.dumps([{"rank": r+1, "score": f"{c.get('rerank_score', 0):.4f}", "law_name": c.get('law_name'), "id": c.get('id')} for r, c in enumerate(final_candidates)], ensure_ascii=False, indent=2)
            }
            all_results.append(result_data)
            
            print(f"\n[LLM 최종 분석 결과]\n{final_answer}")

        except Exception as e:
            print(f"❌ 기사 {i+1} 처리 중 심각한 오류 발생: {e}")
            traceback.print_exc()
            all_results.append({"original_article": content, "llm_analysis": f"처리 중 오류 발생: {e}"})
        finally:
            print(f"\n{'='*30} 기사 {i+1} 분석 종료 {'='*30}")

        # --- 4. 중간 저장 ---
        if (i + 1) % 10 == 0:
            try:
                pd.DataFrame(all_results).to_csv(temp_output_path, index=False, encoding='utf-8-sig')
                print(f"\n✅ {i + 1}개 분석 완료. '{temp_output_path}'에 임시 저장 완료.")
            except Exception as e:
                print(f"❌ 임시 저장 중 오류 발생: {e}")

    # --- 5. 최종 결과 저장 ---
    print("\n--- 모든 기사 분석 완료. 최종 결과를 CSV 파일로 저장합니다. ---")
    try:
        df_results = pd.DataFrame(all_results)
        df_results.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        print(f"\n✅ 분석 결과가 '{output_csv_path}' 파일에 성공적으로 저장되었습니다.")
        if os.path.exists(temp_output_path):
            os.remove(temp_output_path)
            print(f"✅ 임시 파일 '{temp_output_path}'을(를) 삭제했습니다.")
    except Exception as e:
        print(f"\n❌ CSV 파일 저장 중 오류가 발생했습니다: {e}")

        # --- 프로그램 실행 ---

if __name__ == "__main__":
    # ------------------ 설정값 ------------------
    LAW_CSV_PATH = 'law_total.csv'
    NEWS_CSV_PATH = 'error_9.csv'
    OUTPUT_CSV_PATH = 'error_9-RAG_v1.csv' # 결과 파일명 변경
    LLM_MODEL_NAME = 'command-r'
    NUM_ARTICLES_TO_TEST = 3056 # 분석할 기사 수
    # ------------------------------------------

    main(
        law_csv_path=LAW_CSV_PATH,
        news_csv_path=NEWS_CSV_PATH,
        output_csv_path=OUTPUT_CSV_PATH,
        llm_model_name=LLM_MODEL_NAME,
        num_articles_to_test=NUM_ARTICLES_TO_TEST
    )

ModuleNotFoundError: No module named 'sentence_transformers'